## Exercise in Photogrammetry II
# Ex. 01 Relative Orientation
### Submission: xx.xx.xxxx 
### Points: 24

## A Fundamental matrix basics

In the lecture, you were introduced to the concept of the fundamental matrix. Complete the exercise below to strengthen your understanding.

**DATA:** Extract the data from the provided zip file with the password: photo

**Tasks:**

1.  You are given the position and orientation of two *ideal* cameras in world coordinate frame $(c=1)$. Position of camera $C_1$ is $X_0^1=[0,0,0]^T$ and of camera $C_2$ is $X_0^2=[2,5,0]^T$. The cameras viewing directions are along z-axis. The orientation of the $C_1$ and $C_2$ are provided as ```R_1``` and ```R_2```. Construct the fundamental matrix F_12. (2 Points)


In [23]:
import numpy as np
import scipy.linalg
%matplotlib notebook
R_1 = np.eye(3)
R_2 = np.array([[np.sqrt(0.5),-np.sqrt(0.5),0],[np.sqrt(0.5),np.sqrt(0.5),0],[0,0,1]])

print('R_1:',R_1)
print('R_2:',R_2)

R_1: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
R_2: [[ 0.70710678 -0.70710678  0.        ]
 [ 0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]]


In [9]:
# Assumption Ideal camera:
def kMatIdeal(kCamera):
    return np.array([[kCamera, 0, 0], [0, kCamera, 0], [0, 0, 1]])

def skew_mat(vec):
    # takes a vector of length 3 and creates a skewed matrix
    skew = np.array([[0, -vec[2], vec[1]], [vec[2], 0, -vec[0]], [-vec[1], vec[0], 0]])
    return skew
    
def fMatrix(calibCam1, pPos1, rotMat1, calibCam2, pPos2, rotMat2):
    bVecSkew = skew_mat((pPos2 - pPos1).squeeze())
    calibInv1 = np.linalg.inv(calibCam1)
    calibInv2 = np.linalg.inv(calibCam2)
    return calibInv1 @ rotMat1 @ bVecSkew @ rotMat2 @ calibInv2   
    

In [10]:
kMatrix = kMatIdeal(1)
kX0 = np.zeros(3)
kX1 = np.array([2, 5, 0])
f_12 = fMatrix(kMatrix, kX0, R_1, kMatrix, kX1, R_2)

print(np.around(f_12, 2))

[[ 0.    0.    5.  ]
 [ 0.    0.   -2.  ]
 [-2.12  4.95  0.  ]]


2. The bilinear form $x'^TFx'' = 0$ encodes the coplanarity constraint and can be simplified to a matrix multiplication using Kronecker product $\otimes$. Show that $x'^TFx'' = 0$, $F\in \mathbb{R}^{3\times3}$ is equivalent to $(x''\otimes x')^Tf$, where $f \in \mathbb{R}^{9\times1}$. You can either use markdown or plot an image of some handwritten notes for example. (4 Points)

## B Epipolar geometry

You are given a fundamental matrix $F_{12}$ between the images $I_1$ and $I_2$ originated from camera $C_1$ and $C_2$ respectively.

4. How to estimate fo a given point e'' in $I_2$ if it is the epipole of the camera $C_1$? (1 Point)
5. Similarly, how to check if e' is an epipole in $I_1$? (1 Point)
6. Estimate the image coordinates of both epipoles e' and e''. (4 Points) 

The fundamental matrix $F_{12}$ consists of the following values:

In [33]:
F_12 = np.array([[-2,-4,12],
                [6,-2,-8],
                [8,-4,-8]])

epi1 = scipy.linalg.null_space(F_12.T).squeeze()
epi1_img = epi1/epi1[-1]
epi2 = scipy.linalg.null_space(F_12).squeeze()
epi2_img = epi2/epi2[-1]
print("epipole 1 is null space of F.T: {}".format(np.round(epi1_img,2))) 
print("epipole 2 is null space of F: {}".format(np.round(epi2_img, 2)))

epipole 1 is null space of F.T: [-0.29 -1.43  1.  ]
epipole 2 is bull space of F: [2. 2. 1.]


## C Seeing epipolar lines
In this exercise you will learn to visualize the epipolar lines.

7. Use the image point correspondences from the exercise A and the fundamental matrix to write a code that visualizes for every image point from image $I_1$ the corresponding epipolar lines on the image $I_2$. Plot also the correspondend points in $I_2$ (8 Points)

In [74]:
import matplotlib.image as mi
import matplotlib.pyplot as plt
data = np.loadtxt("./data/correspondences.txt")
pts1 = data[:,:2]
pts2 = data[:,2:]

img1 = mi.imread("./data/sydney_opera_1.jpg")
img2 = mi.imread("./data/sydney_opera_2.jpg")
F_sydney = np.loadtxt("./data/F_sydney.txt")

intercepts = []
def pointFrmLines(line1, line2):
    inter = np.cross(line1, line2),
    inter_HC = inter / inter[-1]
    return inter_HC

def epipoleSVD(M):
    return scipy.linalg.null_space(M)

pole1 = epipoleSVD(F_sydney.T).squeeze()
pole1 = pole1/pole1[-1]
print("epipole 1: {}".format(np.round(pole1, 1))) 

pole2 = epipoleSVD(F_sydney).squeeze()
pole2 = pole2/pole2[-1]
print("epipole 2: {}".format(np.round(pole2, 1))) 

l1, l2 = np.zeros((pts1.shape[0], 3)), np.zeros((pts1.shape[0], 3))
for idx in range(pts1.shape[0]):
    pt1 = np.append(pts1[idx, :], [1]).T
    pt2 = np.append(pts2[idx, :], [1]).T
    
    l1[idx,:] = F_sydney @ pt2
    l2[idx,:] = F_sydney.T @ pt1   

epipole 1: [1.8487e+03 6.2180e+02 1.0000e+00]
epipole 2: [7.6538e+03 7.1260e+02 1.0000e+00]


In [ ]:
# draw lines
def linesDef(lines, pole1):
    

## D Epipolar geometry for the normal case

In the lecture you were introduced to the epipolar geometry for the general case. In this exercise your task is to adapt the general formulation to the stereo normal case.

8. Draw the epipolar geometry with all the neccessary variables. You can draw it by hand or your prefered tools. (4 Points)

